# Personality prediction model 
This notebook is used to train a model to predict the personality of a user. For see the results of each line, see the archive network on the folder draft.

## Data management

In [354]:
# Importing libraries 

import pandas as pd 
import numpy as np
import tensorflow as tf
from pathlib import Path
import os

In [355]:
tf.__version__

'2.7.0'

In [356]:
place = str (Path(os.getcwd())) + '\\data\\data.csv' # finding database
data = pd.read_csv(place, sep='\t')                  # opening database

In [357]:
df = data.copy()                                    # copying database

In [358]:
df.drop(['dateload'], axis=1, inplace=True)       # dropping dateload column

In [359]:
filt = ~( (df['IE'] == 0 ) | (df['IE'] == 3) | (df['gender'] == 0) )        # Filter for remove invalid rows in column IE and gender

In [360]:
df = df[filt]                           # Applying filter 

In [361]:
# Optimizing DataFrame

intcolumns = df.columns.drop('country')
df[intcolumns] = df[intcolumns].astype(np.int8)
df['country'] = df['country'].astype('category')

## Data preprocessing

In [362]:
# Label Encoding the "country" and "IE" column

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['country'] = le.fit_transform(df['country'].values)
df['IE'] = le.fit_transform(df['IE'].values)


In [363]:
# Cutting data in parameters and target

X = df.iloc[:, 0:280].values        # all columns except the last one
y = df.iloc[:,280].values           # getting the last column(target)

In [364]:
# Splitting the data into training and test sets

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) 

In [365]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Neural network

In [366]:
ann = tf.keras.models.Sequential()   # Initializing the ANN

In [367]:
ann.add(tf.keras.layers.Dense(units=3, activation='relu')) # Adding the input layer

In [368]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # Adding the first hidden layer

In [369]:
ann.add(tf.keras.layers.Dense(units=3, activation='relu')) # Adding the second hidden layer

In [370]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # Adding the output layer

In [371]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # compiling the ANN

In [372]:
ann.fit(X_train, y_train, batch_size = 16, epochs = 50)

Epoch 1/50
267/267 [==============================] - 1s 2ms/step - loss: 0.4129 - accuracy: 0.8173
Epoch 2/50
267/267 [==============================] - 0s 2ms/step - loss: 0.2290 - accuracy: 0.9176
Epoch 3/50
267/267 [==============================] - 0s 1ms/step - loss: 0.1669 - accuracy: 0.9371
Epoch 4/50
267/267 [==============================] - 0s 1ms/step - loss: 0.1432 - accuracy: 0.9450
Epoch 5/50
267/267 [==============================] - 0s 2ms/step - loss: 0.1295 - accuracy: 0.9490
Epoch 6/50
267/267 [==============================] - 0s 1ms/step - loss: 0.1192 - accuracy: 0.9500
Epoch 7/50
267/267 [==============================] - 0s 1ms/step - loss: 0.1115 - accuracy: 0.9566
Epoch 8/50
267/267 [==============================] - 0s 1ms/step - loss: 0.1060 - accuracy: 0.9591
Epoch 9/50
267/267 [==============================] - 0s 2ms/step - loss: 0.1016 - accuracy: 0.9608
Epoch 10/50
267/267 [==============================] - 0s 1ms/step - loss: 0.0942 - accuracy: 0.9643

## Prediction test

In [373]:
y_pred = ann.predict(X_test)
y_pred

array([[9.5366389e-01],
       [1.5626024e-25],
       [6.2381238e-02],
       ...,
       [7.1459476e-07],
       [9.3593170e-18],
       [6.2381238e-02]], dtype=float32)

In [374]:
y_pred  = (y_pred > 0.5)
y_pred

array([[ True],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [375]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)   # ACCURACY IN ~= 92.71%

[[834  43]
 [ 40 148]]
